In [14]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext

conf = SparkConf()\
    .setMaster("local[*]")\
    .setAppName("Dealing with skewd data")
    # .setExecutorEnv("spark.driver.memory","2g")\
    # .setExecutorEnv("spark.executor.memory","4g")

spark = SparkSession\
    .builder\
    .config(conf=conf)\
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Dealing with skewd data>

In [15]:
import numpy as np
import random

key_1 = ['a'] * 10
key_2 = ['b'] * 600000
key_3 = ['c'] * 800
key_4 = ['d'] * 100
keys = key_1 + key_2 + key_3 + key_4
random.shuffle(keys)


values_1 = list(np.random.randint(low = 1, high = 100, size = len(key_1)))
values_2 = list(np.random.randint(low = 1, high = 100, size = len(key_2)))
values_3 = list(np.random.randint(low = 1, high = 100, size = len(key_3)))
values_4 = list(np.random.randint(low = 1, high = 100, size = len(key_4)))
values = values_1 + values_2 + values_3 + values_4


pair_skew = list(zip(keys,values))

In [16]:
# load data into RDD
rdd = sc.parallelize(pair_skew, 8)

In [17]:
# Generate normal data

key_1 = ['a'] * 5
key_2 = ['b'] * 60
key_4 = ['c'] * 100

keys = key_1 + key_2 + key_4
random.shuffle(keys)


values_1 = list(np.random.randint(low = 1, high = 100, size = len(key_1)))
values_2 = list(np.random.randint(low = 1, high = 100, size = len(key_2)))
values_4 = list(np.random.randint(low = 1, high = 100, size = len(key_4)))
values = values_1 + values_2 + values_4

pair_data = list(zip(keys,values))

In [18]:
small_rdd = sc.parallelize(pair_data, 2)

In [8]:
# Join without salting

rdd_join = rdd.join(small_rdd)
rdd_join.map(lambda x: int(x[1][0] + x[1][1])).reduce(lambda a,b: a+b)

3536160850

In [23]:
rdd_join.take(5)

[('b', (23, 53)),
 ('b', (23, 12)),
 ('b', (23, 81)),
 ('b', (23, 44)),
 ('b', (23, 11))]

In [21]:
rdd.take(5)

[('b', 23), ('b', 87), ('b', 30), ('b', 93), ('b', 4)]

In [22]:
small_rdd.take(5)

[('c', 35), ('c', 32), ('b', 53), ('b', 12), ('c', 27)]

In [12]:
rdd.count()

6010810

In [13]:
small_rdd.count()

165

In [19]:
def salting(val):
    return val + '_' + str(random.randint(0,5))

In [20]:
#import random
# add a random value to the key --> (key, randint)
rdd_new = rdd.map(lambda x: ((x[0], random.randint(0, 10)), x[1]))

# replicate the small data
small_rdd_new = small_rdd.cartesian(sc.parallelize(range(0, 11))).map(lambda x: ((x[0][0], x[1]), x[0][1]))

In [21]:
# Join with salting

rdd_join = rdd_new.join(small_rdd_new)

In [1]:
rdd_join.map(lambda x: x[1][0] + x[1][1]).reduce(lambda a,b: a+b)